In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/recycling waste

/content/drive/MyDrive/recycling waste


In [ ]:
import tensorflow
print(tensorflow.__version__)

2.15.0


In [ ]:

import cv2,os
data_path='/content/drive/MyDrive/recycling waste/'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary
print(label_dict)
print(categories)
print(labels)

{'glass': 0, 'light blubs': 1, 'organic': 2, 'clothes': 3, 'batteries': 4, 'paper': 5, 'e-waste': 6, 'plastic': 7, 'metal': 8}
['glass', 'light blubs', 'organic', 'clothes', 'batteries', 'paper', 'e-waste', 'plastic', 'metal']
[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [ ]:
img_size=100
data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)

    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the image  into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)
        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass t

In [ ]:
import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,3))
target=np.array(target)
from keras.utils import to_categorical
new_target=to_categorical(target)

In [ ]:
new_target.shape

(1298, 9)

In [ ]:
data.shape

(1298, 100, 100, 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 50 neurons
model.add(Dense(9,activation='softmax'))
#The Final layer with 9 outputs for 9 categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 100)       2800      
                                                                 
 activation (Activation)     (None, 98, 98, 100)       0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 100)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 100)       90100     
                                                                 
 activation_1 (Activation)   (None, 47, 47, 100)       0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 100)       0         
 g2D)                                                   

In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

In [ ]:
test_target.shape

(130, 9)

In [ ]:
#checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=50,validation_split=0.2)

Epoch 1/200
30/30 [==============================] - 101s 3s/step - loss: 2.1769 - accuracy: 0.2441 - val_loss: 2.0479 - val_accuracy: 0.2692
Epoch 2/200
30/30 [==============================] - 90s 3s/step - loss: 2.0384 - accuracy: 0.2655 - val_loss: 2.0277 - val_accuracy: 0.2692
Epoch 3/200
30/30 [==============================] - 90s 3s/step - loss: 1.9853 - accuracy: 0.2623 - val_loss: 2.0113 - val_accuracy: 0.2735
Epoch 4/200
30/30 [==============================] - 86s 3s/step - loss: 1.9406 - accuracy: 0.2752 - val_loss: 1.9725 - val_accuracy: 0.2692
Epoch 5/200
30/30 [==============================] - 87s 3s/step - loss: 1.8634 - accuracy: 0.2901 - val_loss: 2.0352 - val_accuracy: 0.2778
Epoch 6/200
30/30 [==============================] - 89s 3s/step - loss: 1.8238 - accuracy: 0.3169 - val_loss: 2.0083 - val_accuracy: 0.2521
Epoch 7/200
30/30 [==============================] - 89s 3s/step - loss: 1.6686 - accuracy: 0.4004 - val_loss: 1.9669 - val_accuracy: 0.2821
Epoch 8/200


In [ ]:
model.save('new_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.preprocessing import image
output_class = ["batteries", "clothes", "e-waste", "glass", "light blubs", "metal", "organic", "paper", "plastic"]
def waste_prediction(new_image):
  test_image = image.load_img(new_image, target_size = (100,100))
  plt.axis("off")
  plt.imshow(test_image)
  plt.show()

  test_image = image.img_to_array(test_image) / 255
  test_image = np.expand_dims(test_image, axis=0)

  predicted_array = model.predict(test_image)
  predicted_value = output_class[np.argmax(predicted_array)]
  predicted_accuracy = round(np.max(predicted_array) * 100, 2)

  print("Your waste material is ", predicted_value, " with ", predicted_accuracy, " % accuracy")